<a href="https://colab.research.google.com/github/namita0210/german_to_english_machine_translation/blob/main/German_to_English_Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import string
from numpy import array
from pickle import dump
from unicodedata import normalize
from pickle import load
from pickle import dump
from numpy.random import rand
from numpy.random import shuffle
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data = '/content/drive/MyDrive/deu.txt'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#function to load the file and preserve the unicode german characters
def load_file(filename):
  file = open(filename , 'r', encoding='utf-8')
  text = file.read()
  file.close()
  return text

In [ ]:
text=load_file(data)
print(text[:99])

Hi.	Hallo!
Hi.	Grüß Gott!
Run!	Lauf!
Wow!	Potzdonner!
Wow!	Donnerwetter!
Fire!	Feuer!
Help!	Hilfe!



In [ ]:
#split the text by phrases
def to_phrase(doc):
  lines = doc.strip().split('\n')
  phrases =[line.split('\t') for line in lines]
  return phrases

In [ ]:
phrases=to_phrase(text)
print(phrases[:3])
print(type(phrases))

[['Hi.', 'Hallo!'], ['Hi.', 'Grüß Gott!'], ['Run!', 'Lauf!']]
<class 'list'>


In [ ]:
string.printable

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [ ]:
def clean(lines):
    cleaned = []
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    table = str.maketrans('','', string.punctuation)

    for l in lines:
        clean_pair=[]
        for x in l :
            x = normalize('NFD',x).encode('ascii','ignore')
            x = x.decode('UTF-8')
            x = x.split()
            x = [word.lower() for word in x]
            x = [word.translate(table) for word in x]
            x = [re_print.sub('' , word) for word in x]
            x = [word for word in x if word.isalpha()]
            clean_pair.append(' '.join(x))
        cleaned.append(clean_pair)
    return array(cleaned)

In [ ]:
# save a list of clean sentences to file
def save_clean_data(sentences, filename):
  dump(sentences, open(filename, 'wb'))
  print('Saved: %s' % filename)

In [ ]:
doc = load_file(filename=data)
pairs = to_phrase(doc)
clean_pairs = clean(pairs)

In [ ]:
clean_pairs

array([['hi', 'hallo'],
       ['hi', 'gru gott'],
       ['run', 'lauf'],
       ...,
       ['if someone who doesnt know your background says that you sound like a native speaker it means they probably noticed something about your speaking that made them realize you werent a native speaker in other words you dont really sound like a native speaker',
        'wenn jemand der deine herkunft nicht kennt sagt dass du wie ein muttersprachler sprichst bedeutet das dass man wahrscheinlich etwas an deiner sprechweise bemerkt hat das erkennen lie dass du kein muttersprachler bist mit anderen worten du horst dich nicht wirklich wie ein muttersprachler an'],
       ['if someone who doesnt know your background says that you sound like a native speaker it means they probably noticed something about your speaking that made them realize you werent a native speaker in other words you dont really sound like a native speaker',
        'wenn jemand fremdes dir sagt dass du dich wie ein muttersprachler 

In [ ]:
save_clean_data(clean_pairs, 'ENGLISH-GERMAN.pkl') # Run Only Once.

Saved: ENGLISH-GERMAN.pkl


In [ ]:
#train-test-split

def load_clean_sentences(filename):
 return load(open(filename, 'rb'))

raw_dataset = load_clean_sentences('ENGLISH-GERMAN.pkl')

In [ ]:
n_sentences = 10000
dataset = raw_dataset[:n_sentences, :]
shuffle(dataset)
train, test = dataset[:9000], dataset[9000:]
save_clean_data(dataset, 'EG-BOTH.pkl')
save_clean_data(train, 'EG-TRAIN.pkl')
save_clean_data(test, 'EG-TEST.pkl')

Saved: EG-BOTH.pkl
Saved: EG-TRAIN.pkl
Saved: EG-TEST.pkl


In [ ]:
#Train Neural Translation model
#Start by loading the train - test data stored in pkl files
# Step 7 : Load the pkl files

dataset = load_clean_sentences('EG-BOTH.pkl')
train = load_clean_sentences('EG-TRAIN.pkl')
test = load_clean_sentences('EG-TEST.pkl')

In [ ]:
#We can use the Keras Tokenize class to map words to integers, as needed for modeling.
#We will use separate tokenizer for the English sequences and the German sequences.
#The function below-named create_tokenizer() will train a tokenizer on a list of phrases
#Step 8 : Create tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [ ]:
# Find the length of longest sequence in the list of phrases
# max sentence length
def max_length(lines):
 return max(len(line.split()) for line in lines)

In [ ]:
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))

English Vocabulary Size: 2404
English Max Length: 5


In [ ]:
# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 0])
print('German Vocabulary Size: %d' % ger_vocab_size)
print('German Max Length: %d' % (ger_length))

German Vocabulary Size: 3856
German Max Length: 5


In [ ]:
# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
 X = tokenizer.texts_to_sequences(lines)
 X = pad_sequences(X, maxlen=length, padding='post')
 return X

In [ ]:
# one hot encode target sequence
def encode_output(sequences, vocab_size):
	ylist = list()
	for sequence in sequences:
		encoded = to_categorical(sequence, num_classes=vocab_size)
		ylist.append(encoded)
	y = array(ylist)
	y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
	return y

In [ ]:
# prepare training data
trainX = encode_sequences(ger_tokenizer, ger_length, train[:, 1])
trainY = encode_sequences(eng_tokenizer, eng_length, train[:, 0])
trainY = encode_output(trainY, eng_vocab_size)
# prepare validation data
testX = encode_sequences(ger_tokenizer, ger_length, test[:, 1])
testY = encode_sequences(eng_tokenizer, eng_length, test[:, 0])
testY = encode_output(testY, eng_vocab_size)

In [ ]:
# define NMT model
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
	model = Sequential()
	model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
	model.add(LSTM(n_units))
	model.add(RepeatVector(tar_timesteps))
	model.add(LSTM(n_units, return_sequences=True))
	model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
	return model

In [ ]:
# define model
model = define_model(ger_vocab_size, eng_vocab_size, ger_length, eng_length, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [ ]:
# summarize defined model
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 5, 256)            987136    
                                                                 
 lstm_2 (LSTM)               (None, 256)               525312    
                                                                 
 repeat_vector_1 (RepeatVec  (None, 5, 256)            0         
 tor)                                                            
                                                                 
 lstm_3 (LSTM)               (None, 5, 256)            525312    
                                                                 
 time_distributed_1 (TimeDi  (None, 5, 2404)           617828    
 stributed)                                                      
                                                                 
Total params: 2655588 (10.13 MB)
Trainable params: 265

In [ ]:
# fit model
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit(trainX, trainY, epochs=30, batch_size=64, validation_data=(testX, testY), callbacks=[checkpoint], verbose=2)

Epoch 1/30

Epoch 1: val_loss improved from inf to 3.68061, saving model to model.h5
141/141 - 33s - loss: 4.4289 - val_loss: 3.6806 - 33s/epoch - 233ms/step
Epoch 2/30


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 3.68061 to 3.55321, saving model to model.h5
141/141 - 21s - loss: 3.5508 - val_loss: 3.5532 - 21s/epoch - 149ms/step
Epoch 3/30

Epoch 3: val_loss improved from 3.55321 to 3.45684, saving model to model.h5
141/141 - 20s - loss: 3.4023 - val_loss: 3.4568 - 20s/epoch - 138ms/step
Epoch 4/30

Epoch 4: val_loss improved from 3.45684 to 3.36912, saving model to model.h5
141/141 - 20s - loss: 3.2528 - val_loss: 3.3691 - 20s/epoch - 140ms/step
Epoch 5/30

Epoch 5: val_loss improved from 3.36912 to 3.27230, saving model to model.h5
141/141 - 21s - loss: 3.1246 - val_loss: 3.2723 - 21s/epoch - 151ms/step
Epoch 6/30

Epoch 6: val_loss improved from 3.27230 to 3.19573, saving model to model.h5
141/141 - 21s - loss: 3.0073 - val_loss: 3.1957 - 21s/epoch - 147ms/step
Epoch 7/30

Epoch 7: val_loss improved from 3.19573 to 3.09409, saving model to model.h5
141/141 - 28s - loss: 2.8749 - val_loss: 3.0941 - 28s/epoch - 197ms/step
Epoch 8/30

Epoch 8: val_loss improved 

Evaluate

In [ ]:
# load datasets
dataset = load_clean_sentences('EG-BOTH.pkl')
train = load_clean_sentences('EG-TRAIN.pkl')
test = load_clean_sentences('EG-TEST.pkl')
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])
# prepare data
trainX = encode_sequences(ger_tokenizer, ger_length, train[:, 1])
testX = encode_sequences(ger_tokenizer, ger_length, test[:, 1])

In [ ]:
# load model
model = load_model('model.h5')

In [ ]:
translate = model.predict()

In [ ]:
# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

In [ ]:
# generate target given source sequence
def predict_sequence(model, tokenizer, source):
	prediction = model.predict(source, verbose=0)[0]
	integers = [argmax(vector) for vector in prediction]
	target = list()
	for i in integers:
		word = word_for_id(i, tokenizer)
		if word is None:
			break
		target.append(word)
	return ' '.join(target)

In [ ]:
# evaluate the skill of the model
def evaluate_model(model, tokenizer, sources, raw_dataset):
	actual, predicted = list(), list()
	for i, source in enumerate(sources):
		# translate encoded source text
		source = source.reshape((1, source.shape[0]))
		translation = predict_sequence(model, eng_tokenizer, source)
		raw_target, raw_src = raw_dataset[i]
		if i < 10:
			print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
		actual.append([raw_target.split()])
		predicted.append(translation.split())
	# calculate BLEU score
	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))